# 3_Insights

In this notebook, I will make all the calculations needed to take the relevant isights for the presentation

# Import libraries
Import the necessary libraries that we are going to use further on.

In [11]:
import pandas as pd
import function_utils as futils

# Import Dataset
Import the dataset from previous analysis

In [12]:
# Read the pickle file 'df_extract.pkl' and store the data in a DataFrame called df
df = pd.read_pickle('data_extract.pkl')

# Create a copy of the df DataFrame and assign it to extract_df
extract_df = df.copy()

# Display the first 5 rows of the case_study_df DataFrame
display(extract_df.head(5))

,vehicle_ID,selling_country,buyer_country,make,model,body_type,fuel_type,colour,registration_date,selling_date,...,deviation_b2B,deviation_b2C,price_to_mileage_ratio,leasing_segment,mileage_segment,selling_price_segment,conversion_rate_segment,b2b_price_segment,b2c_price_segment,stock_turnover_segment
0,V000001,IT,IT,Toyota,4Runner,SUV,Electric,Blue,2023-12-16,2024-12-15,...,0.146179,-0.064134,0.106849,Non-Leasing,>=10.001 km,<= 15.000 €,>=92.01 %,<= 15.000 €,15.001 - 25.000 €,<= 40 days
1,V000002,PT,PT,Chevrolet,Equinox,SUV,Diesel,Silver,2020-05-11,2024-05-10,...,0.122811,-0.124214,0.084335,Non-Leasing,>=10.001 km,<= 15.000 €,82.01 - 88.00 %,<= 15.000 €,<= 15.000 €,51 - 60 days
2,V000003,ES,ES,Chevrolet,Equinox,SUV,Electric,Black,2023-03-21,2024-03-20,...,0.121872,-0.128519,0.101789,Non-Leasing,>=10.001 km,<= 15.000 €,>=92.01 %,<= 15.000 €,15.001 - 25.000 €,41 - 50 days
3,V000004,FR,FR,Porsche,Boxster,Convertible,CNG,White,2015-10-02,2024-09-29,...,-0.098619,-0.144541,0.106886,Non-Leasing,>=10.001 km,15.001 - 25.000 €,<= 82.00%,15.001 - 25.000 €,15.001 - 25.000 €,>=61 days
4,V000005,DE,DE,Subaru,Impreza,Sedan,CNG,Grey,2014-03-15,2024-03-12,...,0.032977,-0.157264,0.032417,Non-Leasing,>=10.001 km,<= 15.000 €,<= 82.00%,<= 15.000 €,<= 15.000 €,41 - 50 days


In [13]:
# Check basic statistics of all dataset after extraction
extract_df.describe()

,selling_price,conversion_rate,b2b_price,b2c_price,stock_turnover,selling_year,selling_month,selling_day,registration_year,registration_month,registration_day,deviation_b2B,deviation_b2C,price_to_mileage_ratio
count,10002.000000,10002.000000,10002.000000,10002.000000,10002.000000,10002.0,10002.000000,10002.000000,10002.000000,10002.000000,10002.000000,10002.000000,10002.000000,10002.000000
mean,13387.985060,85.837994,12635.318587,15131.768499,50.138972,2024.0,6.560788,15.727255,2018.454609,6.556989,15.664967,0.069013,-0.108960,0.331689
std,9459.319277,8.182873,9219.960468,10794.095925,19.629558,0.0,3.448618,8.894688,2.868896,3.450943,8.873757,0.271725,0.542052,0.676324
min,1111.080000,55.050000,1089.820000,1000.000000,15.000000,2024.0,1.000000,1.000000,2009.000000,1.000000,1.000000,-0.166451,-0.199950,0.005671
25%,6786.610000,80.812500,6359.435000,7675.060000,37.000000,2024.0,4.000000,8.000000,2016.000000,4.000000,8.000000,0.032171,-0.138559,0.050058
50%,10841.255000,86.230000,10152.440000,12195.115000,48.000000,2024.0,7.000000,16.000000,2018.000000,7.000000,16.000000,0.073069,-0.114284,0.115042
75%,17161.997500,91.690000,16041.357500,19324.175000,59.000000,2024.0,10.000000,23.000000,2021.000000,10.000000,23.000000,0.106035,-0.093483,0.308962
max,87741.240000,100.000000,92548.500000,103033.820000,156.000000,2024.0,12.000000,31.000000,2024.000000,12.000000,31.000000,26.500000,54.000000,10.318210


# 1. Caracterization of leasing vs non-leasing segment

In [14]:
# create quarter to analyze variables by quarter
extract_df['selling_quarter'] = extract_df['selling_date'].dt.to_period('Q')

In [15]:
# seting output file for analysis
output_file = 'leasing_segments_analysis.xlsx' 

In [16]:
# set the group variables i want to analyze and put on the output file for analysis
grouping_configurations = [
    (['selling_quarter', 'leasing_segment'], 'Quarterly Count'),
    (['selling_country', 'leasing_segment'], 'Selling Country'),
    (['buyer_country', 'leasing_segment'], 'Buyer Country'),
    (['make', 'leasing_segment'], 'Make'),
    (['make', 'model', 'leasing_segment'], 'Make Model'),
    (['body_type', 'leasing_segment'], 'Body Type'),
    (['fuel_type', 'leasing_segment'], 'Fuel Type'),
    (['colour', 'leasing_segment'], 'Colour Type'),
    (['condition_grade', 'leasing_segment'], 'Condition Grade'),
    (['mileage_segment', 'leasing_segment'], 'Mileage Segment'),
    (['selling_price_segment', 'leasing_segment'], 'Selling Price Segment'),
    (['conversion_rate_segment', 'leasing_segment'], 'Conversion Rate Segment'),
    (['b2b_price_segment', 'leasing_segment'], 'B2B Price Segment'),
    (['b2c_price_segment', 'leasing_segment'], 'B2C Price Segment'),
    (['stock_turnover_segment', 'leasing_segment'], 'Stock Turnover Segment')
]



In [17]:
# Running the function to group and add to excel for every pair of variables i want to group. 
# The function counts the number of veichles by grouping variables
for config in grouping_configurations:
    group_columns, sheet_name = config  # Unpack the tuple
    futils.group_and_write_to_excel(extract_df, group_columns, output_file, sheet_name)
